## VGG16

In [1]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from sklearn.cross_validation import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
K.set_image_dim_ordering('th')
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

train_data = np.load('train.npy')
train_target = np.load('train_target.npy')

x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.25, random_state=32)

Using TensorFlow backend.


In [2]:
input = Input(shape=(3, 64, 64))
model = VGG16(weights=None, input_tensor=input, classes=3)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [3]:
datagen = ImageDataGenerator(rotation_range=0.9, zoom_range=0.5, shear_range=0.3, \
                             horizontal_flip=True, vertical_flip=True)
datagen.fit(train_data)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=15, shuffle=True), steps_per_epoch=len(x_train)/100, \
                    epochs=100, verbose=1, validation_data=(x_val_train, y_val_train))

Epoch 1/100
12/12 [==============================] - 8s - loss: 1.0922 - acc: 0.4889 - val_loss: 1.0842 - val_acc: 0.5255
Epoch 2/100
12/12 [==============================] - 3s - loss: 1.0846 - acc: 0.4667 - val_loss: 1.0739 - val_acc: 0.5255
Epoch 3/100
12/12 [==============================] - 3s - loss: 1.0667 - acc: 0.5278 - val_loss: 1.0618 - val_acc: 0.5255
Epoch 4/100
12/12 [==============================] - 3s - loss: 1.0508 - acc: 0.5389 - val_loss: 1.0508 - val_acc: 0.5255
Epoch 5/100
12/12 [==============================] - 3s - loss: 1.0396 - acc: 0.5444 - val_loss: 1.0417 - val_acc: 0.5255
Epoch 6/100
12/12 [==============================] - 3s - loss: 1.0162 - acc: 0.5889 - val_loss: 1.0326 - val_acc: 0.5255
Epoch 7/100
12/12 [==============================] - 3s - loss: 1.0166 - acc: 0.5585 - val_loss: 1.0264 - val_acc: 0.5255
Epoch 8/100
12/12 [==============================] - 3s - loss: 1.0379 - acc: 0.4778 - val_loss: 1.0238 - val_acc: 0.5255
Epoch 9/100
12/12 [=====

12/12 [==============================] - 3s - loss: 1.0457 - acc: 0.4889 - val_loss: 1.0090 - val_acc: 0.5255
Epoch 68/100
12/12 [==============================] - 3s - loss: 0.9944 - acc: 0.5556 - val_loss: 1.0089 - val_acc: 0.5255
Epoch 69/100
12/12 [==============================] - 3s - loss: 0.9911 - acc: 0.5444 - val_loss: 1.0089 - val_acc: 0.5255
Epoch 70/100
12/12 [==============================] - 3s - loss: 0.9369 - acc: 0.5643 - val_loss: 1.0100 - val_acc: 0.5255
Epoch 71/100
12/12 [==============================] - 3s - loss: 1.0217 - acc: 0.5056 - val_loss: 1.0095 - val_acc: 0.5255
Epoch 72/100
12/12 [==============================] - 3s - loss: 1.0263 - acc: 0.5278 - val_loss: 1.0088 - val_acc: 0.5255
Epoch 73/100
12/12 [==============================] - 3s - loss: 1.0295 - acc: 0.5111 - val_loss: 1.0086 - val_acc: 0.5255
Epoch 74/100
12/12 [==============================] - 3s - loss: 0.9567 - acc: 0.5889 - val_loss: 1.0090 - val_acc: 0.5255
Epoch 75/100
12/12 [=========

In [4]:
test_data = np.load('test.npy')
test_id = np.load('test_id.npy')

output_fc1 = K.function(model.inputs + [K.learning_phase()], [model.layers[-2].output])
output = output_fc1([x_train, 1])
print output[0][0]

[ 0.          0.          0.         ...,  0.          0.          0.00723928]


In [5]:
df = pd.DataFrame(output[0])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0,0,0,0.019706,0.006897,0.001724,0.025483,0,0.003439,0,...,0.001052,0.006156,0,0,0.017234,0.001099,0.016355,0,0,0.007239
1,0,0,0,0.015164,0.004856,0.000000,0.022598,0,0.004207,0,...,0.000455,0.004261,0,0,0.015512,0.001469,0.015956,0,0,0.005862
2,0,0,0,0.018266,0.005599,0.001165,0.024677,0,0.003706,0,...,0.001300,0.005392,0,0,0.016862,0.001160,0.016066,0,0,0.007523
3,0,0,0,0.019746,0.006658,0.002134,0.025250,0,0.003504,0,...,0.000550,0.005797,0,0,0.017596,0.001342,0.015858,0,0,0.007391
4,0,0,0,0.019433,0.006184,0.001643,0.024848,0,0.003546,0,...,0.000988,0.005818,0,0,0.017255,0.001361,0.015946,0,0,0.007246


In [6]:
print set(y_train)

set([0, 1, 2])


In [7]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, log_loss, f1_score

svc = SVC(probability=True)

parameters = {'C': [0.5, 1.0, 2.5], 'gamma': ['auto', 0.2, 0.4, 0.8]}

gridsearch = GridSearchCV(svc, parameters, scoring='log_loss')
gridsearch.fit(output[0], y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.5, 1.0, 2.5], 'gamma': ['auto', 0.2, 0.4, 0.8]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [10]:
output_val = output_fc1([x_val_train, 1])
pred = gridsearch.predict(output[0])
#print log_loss(y_val_train, pred)
print pred

[1 1 1 ..., 1 1 1]


In [32]:
output_test = output_fc1([test_data, 1])
predicts = gridsearch.predict_proba(output_test[0])
print predicts

[[ 0.18343079  0.48786499  0.32870422]
 [ 0.18091738  0.47839055  0.34069208]
 [ 0.18400936  0.49083916  0.32515148]
 ..., 
 [ 0.17979798  0.45734844  0.36285358]
 [ 0.18558624  0.48816302  0.32625074]
 [ 0.18220711  0.47232201  0.34547088]]


In [33]:
df = pd.DataFrame(predicts, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission.csv', index=False)